In [57]:
import mediapipe as mp
import pandas as pd
import json

In [60]:
data_map = pd.read_csv('dataset/ASL_Citizen/splits/train.csv')

In [61]:
with open("dataset/WLASL_v0.3.json", 'r') as file:
    data_map1 = json.load(file)

In [62]:
import re

def remove_numbers_from_string(text):

  return re.sub(r"\d+", "", text)

In [63]:
data_map['Gloss'] = data_map['Gloss'].apply(remove_numbers_from_string).astype('string')

In [64]:
data_map.drop_duplicates(subset=['Gloss'], keep='first', inplace=True)

In [65]:
import cv2

In [66]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [67]:
VIDEOS_DIR = 'dataset/ASL_Citizen/videos'

In [68]:
import os
import json

In [72]:
words = [word["gloss"].upper() for word in data_map1]

In [73]:
selected_words = data_map[data_map["Gloss"].isin(words)]

In [90]:
selected_words.reset_index(inplace=True)

In [95]:
word_coordinates = {}

In [102]:
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=2) as hands:
    for ind, word in selected_words.iloc[511:].iterrows():
        print(ind)
        word_coordinates[word["Gloss"]] = []
        video_path = os.path.join(VIDEOS_DIR, word['Video file'])
        video = cv2.VideoCapture(video_path)
        while video.isOpened():
                ret, frame = video.read()
                if not ret:
                    break

                # Resize the frame to 800x750
                frame = cv2.resize(frame, (800, 750))

                # Convert the frame to RGB
                image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Detections
                results = hands.process(image_rgb)

                if results.multi_hand_landmarks:
                    # Initialize hand coordinates

                    frame_hands = {
                        "Left Hand Coordinates": {},
                        "Right Hand Coordinates": {}
                    }

                        # Process each detected hand
                    for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                        # Get the handedness classification from MediaPipe
                        handedness = results.multi_handedness[idx].classification[0].label
                        hand = hand_landmarks.landmark

                        coordinates = []
                        # Store coordinates and joint index in the hand coordinates list
                        for joint_id, landmark in enumerate(hand):
                            x, y, z = landmark.x, landmark.y, landmark.z
                            frame_hands[f"{handedness} Hand Coordinates"][joint_id] = [x, y]
                    word_coordinates[word["Gloss"]].append(frame_hands)


511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760


In [104]:
with open('coordinates.json', 'w') as fp:
    json.dump(word_coordinates, fp)

In [98]:
ALPHABET_DIR = "dataset/Alphabets"

In [99]:
import string

In [100]:
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands=2) as hands:
    for alp in string.ascii_uppercase:
        image_path = os.path.join(ALPHABET_DIR, alp+".jpg")
        for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            # Get the handedness classification from MediaPipe
            handedness = results.multi_handedness[idx].classification[0].label
            hand = hand_landmarks.landmark
            coordinates = {}
            # Store coordinates and joint index in the hand coordinates list
            for joint_id, landmark in enumerate(hand):
                x, y, z = landmark.x, landmark.y, landmark.z
                coordinates[joint_id] = [x, y]
            word_coordinates[alp] = coordinates

In [101]:
with open('coordinates.json', 'w') as fp:
    json.dump(word_coordinates, fp)